In [1]:
from web3 import Web3
from web3.contract.contract import Contract
import json

provider_url = "wss://polygon-mainnet.g.alchemy.com/v2/sExjDUrCgYRxg1GbFGTBogbdSQtT5d3M"

w3 = Web3(Web3.WebsocketProvider(provider_url))

In [2]:
class Token:
    def __init__(self, address: str, decimals: int, name: str) -> None:
        self.address = address
        self.decimals = decimals
        self.name = name
    
    def get_token_amount(self, amount:int) -> int:
        return amount * 10 ** self.decimals

In [9]:
usdc_token = Token(
    address = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174",
    decimals = 6,
    name = "USDC"
)

weth_token = Token(
    address = "0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619",
    decimals = 18,
    name = "wETH"
)

In [3]:
class MarketInterface:
    def __init__(self) -> None:
        pass

    def get_output_amount(self, input_amount:int, input_token: Token, output_token: Token) -> float:
        raise NotImplemented

    def swap(self, input_token: Token, output_token: Token) -> bool:
        raise NotImplemented

    def get_current_price(self, input_token: Token, output_token: Token) -> float:
        raise NotImplemented

In [45]:
class UNIV2Exchange(MarketInterface):

    def __init__(
        self,
        factory_address: str,
        factory_abi: list,
        router_address: str,
        router_abi: list,
        pair_abi: list,
         ):
        super().__init__()
        self.router_contract: Contract = w3.eth.contract(address=router_address, abi=router_abi)
        self.factory_contract: Contract = w3.eth.contract(address=factory_address, abi=factory_abi)
        self.pair_abi = pair_abi


    def get_output_amount(self, input_amount:int, input_token: Token, output_token: Token) -> float:
        #function getAmountsOut(uint amountIn, address[] memory path) public view returns (uint[] memory amounts);
        hex_address = [
            w3.to_checksum_address(input_token.address),
            w3.to_checksum_address(output_token.address),
        ]
        amounts_out = self.router_contract.functions.getAmountsOut(input_amount, hex_address).call()
        return amounts_out[-1] / (10 ** output_token.decimals)


    def swap(self, input_token: Token, output_token: Token) -> bool:
        pass

    def get_current_price(self, base_token: Token, quote_token: Token) -> float:
        # Precio = Reservas del base token / Reservas del quote token
        # Pools UNIV2 (AMM de producto constante)

        #function getPair(address tokenA, address tokenB) external view returns (address pair);
        pair_address = self.factory_contract.functions.getPair(base_token.address, quote_token.address).call()
        pair_contract = w3.eth.contract(address=pair_address, abi=self.pair_abi)

        # function token0() external view returns (address);
        token0_address = pair_contract.functions.token0().call()
        # function token1() external view returns (address);
        token1_address = pair_contract.functions.token1().call()
        # function getReserves() external view returns (uint112 reserve0, uint112 reserve1, uint32 blockTimestampLast);

        reserves = pair_contract.functions.getReserves().call()
        token_0_reserves = reserves[0]
        token_1_reserves = reserves[1]
        
        if token0_address == base_token.address:
            base_token_reserves = token_0_reserves / 10 ** base_token.decimals
            quote_token_reserves = token_1_reserves / 10 ** quote_token.decimals
        else:
            base_token_reserves = token_1_reserves / 10 ** base_token.decimals
            quote_token_reserves = token_0_reserves / 10 ** quote_token.decimals
            
        return base_token_reserves / quote_token_reserves

In [46]:
sushi_exchange = UNIV2Exchange(
    factory_address = "0xc35DADB65012eC5796536bD9864eD8773aBc74C4",
    factory_abi = json.load(open("factory_abi.json")),
    router_abi = json.load(open("router_abi.json")),
    router_address="0x1b02dA8Cb0d097eB8D57A175b88c7D8b47997506",
    pair_abi = json.load(open("pair_abi.json"))
)


print(f"Sushiswap get_amount_out: {sushi_exchange.get_output_amount(weth_token.get_token_amount(1), weth_token, usdc_token)}")
print(f"Sushiswap get_price: {sushi_exchange.get_current_price(usdc_token, weth_token)}")

Sushiswap get_amount_out: 1784.011527
Sushiswap get_price: 1798.127233480204


In [49]:
quickswap_exchange = UNIV2Exchange(
    factory_address = "0x5757371414417b8C6CAad45bAeF941aBc7d3Ab32",
    factory_abi = json.load(open("factory_abi.json")),
    router_abi = json.load(open("router_abi.json")),
    router_address="0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff",
    pair_abi = json.load(open("pair_abi.json"))
)


print(f"Quickswap get_amount_out: {quickswap_exchange.get_output_amount(weth_token.get_token_amount(1), weth_token, usdc_token)}")
print(f"Quickswap get_price: {quickswap_exchange.get_current_price(usdc_token, weth_token)}")

Quickswap get_amount_out: 1790.865881
Quickswap get_price: 1799.4598503387529


In [51]:
print(f"Sushiswap get_price: {sushi_exchange.get_current_price(usdc_token, weth_token)}")
print(f"Quickswap get_price: {quickswap_exchange.get_current_price(usdc_token, weth_token)}")

Sushiswap get_price: 1798.127233480204
Quickswap get_price: 1799.4598503387529


In [58]:
import time
while True:
    amount_in = weth_token.get_token_amount(1)
    prev_amount_out = sushi_exchange.get_output_amount(amount_in, weth_token, usdc_token)
    amount_out = quickswap_exchange.get_output_amount(int(usdc_token.get_token_amount(prev_amount_out)), usdc_token, weth_token)
    print(f"Initial input: {amount_in}, final output: {amount_out}")

    time.sleep(1)

Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000, final output: 0.9871510958865878
Initial input: 1000000000000000000

KeyboardInterrupt: 

In [60]:
import requests
import json

binance_api_url = "https://api.binance.us"

uri = "/api/v3/depth"

response = requests.get(binance_api_url + uri , {
    "symbol":"ETHUSDC",
    "limit":5
})

if response.status_code == 200:
    data = response.json()
    print(data)

{'lastUpdateId': 277937432, 'bids': [['1790.68000000', '0.04450000'], ['1790.67000000', '2.74050000'], ['1789.39000000', '0.00560000'], ['1786.14000000', '0.01850000'], ['1780.00000000', '0.03500000']], 'asks': [['1815.24000000', '0.04000000'], ['1815.25000000', '0.64680000'], ['1818.36000000', '0.01830000'], ['1825.58000000', '0.00560000'], ['1827.41000000', '0.01820000']]}


In [61]:
uri = "/api/v3/avgPrice"

response = requests.get(binance_api_url + uri , {
    "symbol":"ETHUSDC",
})

if response.status_code == 200:
    data = response.json()
    print(data)

{'mins': 5, 'price': '1816.65000000'}


In [64]:
import time
import hmac
import hashlib

# Set up authentication
API_KEY="vmPUZE6mv9SD5VNHk4HlWFsOr6aKE2zvsw0MuIgwCIPy6utIco14y7Ju91duEh8A"
PRIVATE_KEY="NhqPtmdSJYdKjVHjA7PZj4Mge3R5YNiP1e3UZjInClVN65XAbvqqM6A7H5fATj0j"

params = {
    "symbol" : "LTCBTC",
    'side':         'BUY',
    'type':         'LIMIT',
    'timeInForce':  'GTC',
    'quantity':     '1',
    'price':        '0.1',
    'recvWindow':   '5000',
    'timestamp':    '1499827319559'
}

#timestamp = int(time.time() * 1000) # Timestamp en milliseconds
#params["timestamp"] = timestamp

payload = '&'.join([f"{key}={value}" for key, value in params.items()])

signature = hmac.new(
    bytes(PRIVATE_KEY, 'latin-1'),
    msg = bytes(payload, 'latin-1'),
    digestmod = hashlib.sha256
).hexdigest()

params["signature"] = signature

headers = {
    "X-MBX-APIKEY" : API_KEY
}

response = requests.post(binance_api_url + "/api/v3/order", headers = headers, params = params)

5e93fda387a7edeb4e612e9adb477cd96e5a2b78be0d2fd6e0142a76c3ff7581


In [65]:
#%pip install ccxt

In [66]:
import ccxt

binance_us_exchange = ccxt.binanceus()

In [67]:
binance_us_exchange.fetch_ticker("ETH/USDC")

{'symbol': 'ETH/USDC',
 'timestamp': 1698970243376,
 'datetime': '2023-11-03T00:10:43.376Z',
 'high': 1875.0,
 'low': 1786.14,
 'bid': 1791.5,
 'bidVolume': 0.0444,
 'ask': 1814.17,
 'askVolume': 0.0836,
 'vwap': 1848.92244336,
 'open': 1859.85,
 'close': 1814.54,
 'last': 1814.54,
 'previousClose': 1846.0,
 'change': -45.31,
 'percentage': -2.436,
 'average': 1837.195,
 'baseVolume': 13.1757,
 'quoteVolume': 24360.847437,
 'info': {'symbol': 'ETHUSDC',
  'priceChange': '-45.31000000',
  'priceChangePercent': '-2.436',
  'weightedAvgPrice': '1848.92244336',
  'prevClosePrice': '1846.00000000',
  'lastPrice': '1814.54000000',
  'lastQty': '0.00550000',
  'bidPrice': '1791.50000000',
  'bidQty': '0.04440000',
  'askPrice': '1814.17000000',
  'askQty': '0.08360000',
  'openPrice': '1859.85000000',
  'highPrice': '1875.00000000',
  'lowPrice': '1786.14000000',
  'volume': '13.17570000',
  'quoteVolume': '24360.84743700',
  'openTime': '1698883843376',
  'closeTime': '1698970243376',
  'fir

In [68]:
binance_us_exchange.fetch_order_book("ETH/USDC", 5)

{'symbol': 'ETH/USDC',
 'bids': [[1791.55, 0.0428],
  [1791.53, 2.9395],
  [1789.39, 0.0056],
  [1786.14, 0.0185],
  [1780.0, 0.035]],
 'asks': [[1814.12, 0.0925],
  [1814.14, 2.4367],
  [1818.36, 0.0183],
  [1825.58, 0.0056],
  [1827.41, 0.0182]],
 'timestamp': None,
 'datetime': None,
 'nonce': 277938019}

In [70]:
symbol = ["ETH/USDC"]

window_short = 1
window_long = 2

candles = []
long = False

markets = [binance_us_exchange, ...]

import time

while True:

    start_time = time.time()

    vela = [market.fetch_ohlcv(symbol, "1m", params={
        "limit":1
    })[0] for market in markets]

    candles.append(vela[4]) # Me quedo con el precio de cierre

    short_ma = sum(candles[-window_short:]) / len(candles[-window_short:])
    long_ma = sum(candles[-window_long:]) / len(candles[-window_long:])

    print(f"short_ma: {short_ma}, long_ma: {long_ma}")

    candles = candles[-window_long:]

    if not long:
        if short_ma > long_ma:
            print("compro")
            long = True
    else:
        if short_ma < long_ma:
            print("vendo")
            long = False
    time.sleep(60 - (time.time() - start_time) % 60)


[[1698970500000, 1814.54, 1814.54, 1814.54, 1814.54, 0.0]]


KeyboardInterrupt: 